In [ ]:
# 주피터
import os

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [ ]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

## 필요모듈 import

In [ ]:
!pip install konlpy   # 코랩에서만 실행

In [ ]:
from tqdm import tqdm
import scipy as sp
import pandas as pd
import numpy as np
import re

from konlpy.tag import Kkma    
from konlpy.tag import Hannanum    
from konlpy.tag import Okt        
from konlpy.tag import *
import nltk
import pickle

# from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
from datetime import datetime, timedelta

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [ ]:
kkma = Kkma()
hannanum = Hannanum()
okt = Okt()

## 자연어 데이터 import(날짜 가져오기 위해 먼저 가져옴)

In [ ]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [ ]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [ ]:
naver_talks['Date'] = pd.to_datetime(naver_talks['Date'])
daum_talks['Date'] = pd.to_datetime(daum_talks['Date'])

,Date,Title
0,2021-01-01,새해 첫글이네욤
1,2021-01-01,54만원에매수해 매도중
2,2021-01-01,돈도 안들고 청약하러갔나미련한아지매네그남편누군지피곤하긋다 ㅋㅋ
3,2021-01-01,32만원에 들어간사람도 인정해줍니까
4,2021-01-01,2021 갈아온 발톱을 드러내는 한 해가 될 듯
...,...,...
37016,2022-06-16,삼성 전기차 태스크포크를 발족했다는 소식
37017,2022-06-17,긍정적인 부분은
37018,2022-06-18,삼성 반세기의 혁신을 넘어 100년 대계를 향하다
37019,2022-06-26,숏텀 롱텀 디플레이션 전쟁


In [ ]:
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))     # str(변수) 는 자체 내장함수, 변수 하나씩밖에 적용 안됨.
news_df.sort_values('Date', ignore_index=True, inplace=True)

,Date,Title
0,20210101,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,20210101,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,20210101,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,20210101,삼성전자 신고가 랠리 9만전자 가능할까
4,20210101,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


In [ ]:
news_df[news_df['Date']== '2021-01-04'].head(2)

In [ ]:
news_df = news_df[665:]
news_df.head(3)

## 주가 데이터

In [ ]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

20210101
20220627


### KRX에서 받은 csv로 가져오기

In [ ]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2021/01/04,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530
1,2021/01/05,686000,15000,2.24,675000,686000,658000,601692,403850137943,47172467580000,68764530
2,2021/01/06,680000,-6000,-0.87,693000,702000,671000,669150,457568981000,46759880400000,68764530
3,2021/01/07,698000,18000,2.65,688000,712000,676000,756799,526297188000,47997641940000,68764530
4,2021/01/08,739000,41000,5.87,710000,739000,705000,890113,643077420000,50816987670000,68764530


In [ ]:
# 2진분류
stock_df['updown'] = 0
for i in tqdm(range(len(stock_df))):
    if stock_df['등락률'][i] > 0:
        stock_df['updown'][i] = 1
    else:
        stock_df['updown'][i] = 0
stock_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:05<00:00, 71.33it/s]


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,updown
일자,,,,,,,,,,,
2021-01-04,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-05,686000,15000,2.24,675000,686000,658000,601692,403850137943,47172467580000,68764530,1
2021-01-06,680000,-6000,-0.87,693000,702000,671000,669150,457568981000,46759880400000,68764530,0
2021-01-07,698000,18000,2.65,688000,712000,676000,756799,526297188000,47997641940000,68764530,1
2021-01-08,739000,41000,5.87,710000,739000,705000,890113,643077420000,50816987670000,68764530,1


In [ ]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

100%|████████████████████████████████████████████████████████████████████████| 702485/702485 [02:12<00:00, 5303.05it/s]


,Date,Title
0,2021-01-01 00:00:00,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,2021-01-01 00:00:00,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,2021-01-01 00:00:00,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,2021-01-01 00:00:00,삼성전자 신고가 랠리 9만전자 가능할까
4,2021-01-01 00:00:00,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


## 합치기

In [ ]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률(y)', 'updown']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.head()

,Date,Title,일자
0,2021-01-01 00:00:00,서학개미 사로잡은 美 ESG ETF 올해도 高高할까,2020-12-31
1,2021-01-01 00:00:00,목표가 속속 100만원대 흥 오른 삼성바이오로직스,2020-12-31
2,2021-01-01 00:00:00,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익,2020-12-31
3,2021-01-01 00:00:00,삼성전자 신고가 랠리 9만전자 가능할까,2020-12-31
4,2021-01-01 00:00:00,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다,2020-12-31
...,...,...,...
702480,2022-06-16 00:00:00,삼성 전기차 태스크포크를 발족했다는 소식,2022-06-15
702481,2022-06-17 00:00:00,긍정적인 부분은,2022-06-16
702482,2022-06-18 00:00:00,삼성 반세기의 혁신을 넘어 100년 대계를 향하다,2022-06-17
702483,2022-06-26 00:00:00,숏텀 롱텀 디플레이션 전쟁,2022-06-25


## 감성분석 모델 적용

### 감성사전 만들기

In [ ]:
n_ = []
title_rename = []
for i in tqdm(range(len(df))):
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "",df.iloc[i]['Title']))
    n_.append(' '.join(okt.nouns(df.iloc[i]['Title'])))
df['nouns'] = n_
df['Title']=title_rename
df = df[df['nouns']!='']

100%|█████████████████████████████████████████████████████████████████████████| 416942/416942 [08:06<00:00, 856.35it/s]


In [ ]:
vocab = {}
cnt = 0
for i in tqdm(df['nouns']):
    i = i.split(' ')
    for j in range(len(i)):
        if i[j] in vocab or len(i[j]) <= 1:
            cnt = cnt+1
            pass
        else:
            vocab[i[j]] = 0
print(len(vocab))
print(vocab)

100%|██████████████████████████████████████████████████████████████████████| 414989/414989 [00:01<00:00, 262821.42it/s]

35174
{'고객': 0, '대접': 0, '쿠키': 0, '백화점': 0, '직원': 0, '코스피': 0, '육박': 0, '배경': 0, '탁금': 0, '돌파': 0, '시대': 0, '마켓': 0, '인사이트': 0, '두산': 0, '새옹지마': 0, '전보': 0, '퓨얼': 0, '블록딜': 0, '아시아': 0, '주요': 0, '증시': 0, '동향': 0, '국민연금': 0, '대한항공': 0, '아시아나': 0, '인수': 0, '반대': 0, '주주': 0, '가치': 0, '훼손': 0, '우려': 0, '마감': 0, '종목': 0, '뉴스': 0, '세스코': 0, '음식점': 0, '도움': 0, '기업': 0, '공시': 0, '셀카': 0, '중고차': 0, '오토': 0, '허브': 0, '운영': 0, '동해': 0, '홀딩스': 0, '매각': 0, '뉴욕': 0, '개장': 0, '하락': 0, '출발': 0, '조지아주': 0, '선거': 0, '시작': 0, '속보': 0, '뉴욕증시': 0, '반등': 0, '지수': 0, '상승': 0, '사례': 0, '중심': 0, '강의': 0, '판례': 0, '에듀윌': 0, '공인': 0, '중개사': 0, '민법': 0, '민사': 0, '특별법': 0, '심정욱': 0, '교수': 0, '천스닥': 0, '눈앞': 0, '박셀': 0, '바이오': 0, '투자': 0, '위험': 0, '지정': 0, '거래': 0, '정지': 0, '코로나': 0, '담배': 0, '소비': 0, '역대': 0, '최대': 0, '신년': 0, '예측': 0, '동학': 0, '개미': 0, '어디': 0, '데일리안': 0, '오늘': 0, '종합': 0, '박원순': 0, '피소': 0, '남인순': 0, '맹폭': 0, '배달': 0, '부담': 0, '직접': 0, '장님': 0, '어르신': 0, '나라': 0, '경제력': 0, '여가': 0, '경로당': 0, '규모

In [ ]:
up = 126886
down = 136220
up_ratio = up/(up+down)
down_ratio = down/(up+down)

import collections
for i,w in tqdm(enumerate(df['nouns'])):
    w = w.split(' ')
    if (df.iloc[i]['updown']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] - up_ratio

414989it [00:45, 9127.11it/s] 


In [ ]:
sent_dictionary = vocab
total = []
for i,w  in tqdm(enumerate(df['nouns'])):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
            continue
        elif(j not in sent_dictionary):
            continue
        else:
            sent_score = sent_score + sent_dictionary[j]
    total.append(sent_score/len(w))
df['sent_score'] = total

414989it [00:01, 215517.79it/s]


In [ ]:
## 평균값 이상인 경우 1, 아닌 경우 0
df['sent_label'] = 0
df

,제목,등락률,updown,nouns,sent_score,sent_label
일자,,,,,,
2021-01-04,고객 대접할 쿠키를 발 위에 백화점 직원들의 일탈,6.85,1,고객 대접 쿠키 발 위 백화점 직원 탈,1.771519,0
2021-01-04,코스피 3000선 육박 배경엔 68조 고객예탁금,6.85,1,코스피 선 육박 배경 고객 탁금,-42.632940,0
2021-01-04,코스피 1000 돌파후 31년만에 삼천피 시대 연다,6.85,1,코스피 돌파 후 피 시대 연,2.287201,0
2021-01-04,마켓인사이트 두산 오너家의 새옹지마 석달 전보다 50 높은값에 퓨얼셀 블록딜,6.85,1,마켓 인사이트 두산 너 의 새옹지마 달 전보 값 퓨얼 셀 블록딜,8.672453,0
2021-01-04,표 아시아 주요 증시 동향 1월 5일,6.85,1,표 아시아 주요 증시 동향,-67.860131,0
...,...,...,...,...,...,...
2022-06-29,상반기 투자 성적 기관 외국인 동학개미 서학개미,-1.89,0,상반기 투자 성적 기관 외국인 동학 개미 서학 개미,-45.925364,0
2022-06-29,LG엔솔 美공장 재검토 소식에 배터리 업종 동반 급락,-1.89,0,엔솔 공장 재 검토 소식 배터리 업종 동반 급락,-43.640681,0
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,-1.89,0,하반기 애플 신제품 대거 출시 애플 부품 주 반등 시도,-57.417971,0


In [ ]:
sum = 0
for i in tqdm(range(len(vocab))):
    sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in tqdm(range(len(df))):
    if df['sent_score'][i] > sent_mean:
        a_.append(1)
    else:
        a_.append(0)
df['sent_label'] = a_
df

100%|██████████████████████████████████████████████████████████████████████| 414989/414989 [00:02<00:00, 206331.52it/s]


,제목,등락률,updown,nouns,sent_score,sent_label
일자,,,,,,
2021-01-04,고객 대접할 쿠키를 발 위에 백화점 직원들의 일탈,6.85,1,고객 대접 쿠키 발 위 백화점 직원 탈,1.771519,1
2021-01-04,코스피 3000선 육박 배경엔 68조 고객예탁금,6.85,1,코스피 선 육박 배경 고객 탁금,-42.632940,0
2021-01-04,코스피 1000 돌파후 31년만에 삼천피 시대 연다,6.85,1,코스피 돌파 후 피 시대 연,2.287201,1
2021-01-04,마켓인사이트 두산 오너家의 새옹지마 석달 전보다 50 높은값에 퓨얼셀 블록딜,6.85,1,마켓 인사이트 두산 너 의 새옹지마 달 전보 값 퓨얼 셀 블록딜,8.672453,1
2021-01-04,표 아시아 주요 증시 동향 1월 5일,6.85,1,표 아시아 주요 증시 동향,-67.860131,0
...,...,...,...,...,...,...
2022-06-29,상반기 투자 성적 기관 외국인 동학개미 서학개미,-1.89,0,상반기 투자 성적 기관 외국인 동학 개미 서학 개미,-45.925364,0
2022-06-29,LG엔솔 美공장 재검토 소식에 배터리 업종 동반 급락,-1.89,0,엔솔 공장 재 검토 소식 배터리 업종 동반 급락,-43.640681,0
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,-1.89,0,하반기 애플 신제품 대거 출시 애플 부품 주 반등 시도,-57.417971,0


In [ ]:
df['sent_label'].value_counts()

0    258399
1    156590
Name: sent_label, dtype: int64

In [ ]:
# Mecab 은 colab!

In [ ]:
df['tokenized'] = 0

In [ ]:
# from konlpy.tag import Mecab
# train_data, validation_data = train_test_split(df_train, test_size = 0.2, random_state = 42)

df.dropna(how='any',inplace= True)
# validation_data.dropna(how='any', inplace=True)
df['tokenized'] = df['Title'].apply(okt.morphs)   # 또는 자유롭게 라이브러리 사용
# mecab = Mecab() 
# validation_data['tokenized'] = validation_data['제목'].apply(mecab.morphs)
X_train = df['tokenized'].values
Y_train = df['sent_label'].values
# X_vali= validation_data['tokenized'].values
# Y_vali = validation_data['sent_label'].values


KeyboardInterrupt



In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 빈도 수가 낮은 단어 제거
threshold = 3 # 3개 미만이면 제거

words_cnt = len(tokenizer.word_index)
rare_cnt = 0
words_freq = 0
rare_freq = 0

for key, value in tqdm(tokenizer.word_counts.items()):
    words_freq = words_freq + value

    if value < threshold:
        rare_cnt += 1
        rare_freq = rare_freq + value

print("전체 단어 수 :", words_cnt)
print("빈도가 {} 이하인 희귀 단어 수 : {}".format(threshold-1, rare_cnt))
print("희귀 단어 비율 : {}".format((rare_cnt/words_cnt) * 100))
print("희귀 단어 등장 빈도 비율 : {}".format((rare_freq / words_freq) * 100))

,re제목,sent_label
일자,,
2021-01-04,美 뉴욕증시 S P500지수 장중 최고치 기록하며 2021년 출발,0
2021-01-04,美 S P500지수 장중 최고치 찍으며 새해 출발,0
2021-01-04,새해 첫날부터 최고가 코스피 2 900 넘었다,0
2021-01-04,새해 첫 거래일 코스피 시가총액 사상 첫 2000조원 돌파,0
2021-01-04,증권사 신년사 키워드 디지털 전환 ESG 경영,0
...,...,...
2022-06-30,주식 매매 상위 종목 및 환율 2022년 6월 29일자,0
2022-06-30,유럽증시 경기침체 공포 커지면서 하락,0
2022-06-30,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,0


In [ ]:
vocab_size = words_cnt - rare_cnt + 2
print(vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
# X_vali = tokenizer.texts_to_sequences(X_vali)

In [ ]:
print('리뷰 최대 길이 :', max(len(i) for i in X_train))
# print('리뷰 평균 길이 :', sum(map(len, X_train))/ len(X_train))

In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in tqdm(nested_list):
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 25
below_threshold_len(max_len, X_train)

# 패딩
X_train = pad_sequences(X_train, maxlen = max_len)
# X_vali = pad_sequences(X_vali, maxlen = max_len)

## 모델링-LSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU, LSTM, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

# model = Sequential()
# model.add(Embedding(vocab_size, 30))
# model.add(LSTM(128)) # overfitting -> dropout
# model.add(Dense(1, activation = 'sigmoid'))

# model.compile(optimizer = 'rmsprop',
#               loss = 'binary_crossentropy',
#               metrics = ['acc'])
model.summary()

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\jinyo\\AppData\\Local\\Temp\\pip-uninstall-vn0epspn\\pip.exe'
Consider using the `--user` option or check the permissions.

ERROR: JPype1-0.5.7-cp27-none-win_amd64.whl is not a supported wheel on this platform.


In [ ]:
# 저장된 모델 불러오기, 평가
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(X_train, Y_train)

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
# 시각화

hist_dict = history.history
loss = hist_dict['loss']
val_loss = hist_dict['val_loss']
acc = hist_dict['acc']
val_acc = hist_dict['val_acc']

plt.plot(loss, 'b--', label = 'training loss')
plt.plot(val_loss, 'r--', label = 'validation loss')
plt.legend()
plt.grid()

plt.figure()
plt.plot(acc, 'b--', label = 'training accuracy')
plt.plot(val_acc, 'r--', label = 'validation accuracy')
plt.legend()
plt.grid()

plt.show()

In [ ]:
# 임의 불용어
stopwords1 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords2 = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords = list(set(stopwords1 + stopwords2))

def sentiment_predict(new_sentence):
    new_token = [word for word in mecab.morphs(new_sentence) if not word in stopwords]
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    score = float(loaded_model.predict(new_pad))

    if score > 0.5:
        print("{} -> 긍정 ({:.2f}%)".format(new_sentence, score * 100))
    else:
        print("{} -> 부정 ({:.2f}%)".format(new_sentence, (1-score) * 100))

In [ ]:
sentiment_predict("[유럽개장] 장초반 상승세…英 1.03%↑")
sentiment_predict("[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감")

## vocab 확인

In [ ]:
import pandas as pd 

df = pd.DataFrame([vocab])

In [ ]:
vocab_values = list(vocab.values())
vocab_values

vocab_values2 = []

for i in tqdm(range(len(vocab_values))): 
    vocab_values2.append(round(vocab_values[i],2))
  

In [ ]:
print(max(vocab_values2))
print(min(vocab_values2))

In [ ]:
fig = plt.figure(figsize = (20,20))
plt.plot(range(len(vocab_values2)), vocab_values2)